### Activating KVM on Jetson Nano

This is based on https://developer.ridgerun.com/wiki/index.php?title=Jetson_Nano/Development/Building_the_Kernel_from_Source

Jetson Nano's original image does not come with KVM enabled, thus we have to recompile the kernel and activate it. In this article, we're gonna do everything inside the Nano so we don't have to take out the SD card or flash a new image. You might be surprised that recompiling the kernel in the Nano itself takes less than 30 minutes.

So, put your Nano to work with the latest Ubuntu image provided by Jetson Nano, and then boot it and install the dependencies needed to build the kernel:

Now, we should get the kernel source at [developer.nvidia.com](https://developer.nvidia.com/embedded/downloads). As af today, (july 15) the latest is [r32_release_v5.1/sources/t210](https://developer.nvidia.com/embedded/l4t/r32_release_v5.1/r32_release_v5.1/sources/t210/public_sources.tbz2). But wait, use the script below to download and unpack everything.

#### Documents

- [Getting Started Page](https://developer.nvidia.com/embedded/learn/get-started-jetson-nano-devkit#write): [Jetson Nano Developer Kit SD Card Image 📦](https://developer.nvidia.com/jetson-nano-sd-card-image)
  - [r32_Release_v7.1/JP_4.6.1_b110_SD_Card/Jeston_Nano/jetson-nano-jp461-sd-card-image.zip 📦 (redirect)](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.1/JP_4.6.1_b110_SD_Card/Jeston_Nano/jetson-nano-jp461-sd-card-image.zip)
- [JetPack Archive
](https://developer.nvidia.com/embedded/jetpack-archive)
  - **[JetPack 4.6.1](https://developer.nvidia.com/embedded/jetpack-sdk-461)**: [r32_release_v7.1/jp_4.6.1_b110_sd_card/jeston_nano/jetson-nano-jp461-sd-card-image.zip 📦](https://developer.nvidia.com/embedded/l4t/r32_release_v7.1/jp_4.6.1_b110_sd_card/jeston_nano/jetson-nano-jp461-sd-card-image.zip)
    - **[Jetson Linux R32.7.1](https://developer.nvidia.com/embedded/linux-tegra-r3271)**
      - **L4T Driver Package (BSP) Sources**:
        - [r32_Release_v7.1/Sources/T210/public_sources.tbz2 📦](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.1/Sources/T210/public_sources.tbz2)
  <!-- - **[JetPack SDK 4.6.3](https://developer.nvidia.com/jetpack-sdk-463)**: (JetPack 4.6.3 can be installed using SDK Manager.)
    - **[Jetson Linux R32.7.3](https://developer.nvidia.com/embedded/linux-tegra-r3273)**
      - **Driver Package (BSP) Sources**:
        - [r32_release_v73_sources_t210_public_sources_tbz2 📦](https://developer.nvidia.com/downloads/remack-sdksjetpack-463r32releasev73sourcest210publicsourcestbz2)
  - **[JetPack SDK 4.6.5](https://developer.nvidia.com/jetpack-sdk-465)**
    - **[Jetson Linux R32.7.5](https://developer.nvidia.com/embedded/linux-tegra-r3275)**
      - **Developer Guide**:
        - [NVIDIA Jetson Linux Developer Guide : Introduction (32.7.5)](https://docs.nvidia.com/jetson/archives/l4t-archived/l4t-3275/index.html) -->
  - **[JetPack SDK 4.6.6](https://developer.nvidia.com/jetpack-sdk-466)**: (JetPack 4.6.6 can be installed using SDK Manager.)
    - **[Jetson Linux R32.7.6](https://developer.nvidia.com/embedded/linux-tegra-r3276)** - *final release for Jetson Linux R32 and JetPack 4*
      - **Driver Package (BSP) Sources**:
          <!-- - [r32_release_v7.6/sources/t210/public_sources.tbz2 📦](https://developer.nvidia.com/downloads/embedded/l4t/r32_release_v7.6/sources/t210/public_sources.tbz2) -->
          - [r32_Release_v7.6/sources/T210/public_sources.tbz2 📦](https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.6/sources/T210/public_sources.tbz2)
      - **Developer Guide**: [NVIDIA Jetson Linux Developer Guide : Introduction (32.7.6)](https://docs.nvidia.com/jetson/archives/l4t-archived/l4t-3276/index.html)
        - [NVIDIA Jetson Linux Developer Guide : Kernel Customization (32.7.6)](https://docs.nvidia.com/jetson/archives/l4t-archived/l4t-3276/index.html#page/Tegra%20Linux%20Driver%20Package%20Development%20Guide/kernel_custom.html#)
  - **JetPack SDK Common**:
    - **Jetson Linux Common**:
      - **GCC 7.3.1 for 64 bit BSP and Kernel**:
        - [gcc-linaro-7.3.1-2018.05-x86_64_aarch64-linux-gnu.tar.xz 📦](https://developer.nvidia.com/embedded/dlc/l4t-gcc-7-3-1-toolchain-64-bit)

#### Environments

In [1]:
# 現在動いているSDイメージとカーネルのバージョンを確認する
cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.6 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.6 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In [2]:
cat /etc/nv_tegra_release
# => # R32 (release), REVISION: 7.3, GCID: 31982016, BOARD: t210ref, EABI: aarch64, DATE: Tue Nov 22 17:30:08 UTC 2022

# R32 (release), REVISION: 7.6, GCID: 38171779, BOARD: t210ref, EABI: aarch64, DATE: Tue Nov  5 07:46:14 UTC 2024


In [3]:
cat /proc/version
# => Linux version 4.9.299-tegra (buildbrain@mobile-u64-5333-d8000) (gcc version 7.3.1 20180425 [linaro-7.3-2018.05 revision d29120a424ecfbc167ef90065c0eeb7f91977701] (Linaro GCC 7.3-2018.05) ) #1 SMP PREEMPT Tue Nov 22 09:24:39 PST 2022
# => Linux version 4.9.299-tegra (codespace@codespaces-600304) (gcc version 7.3.1 20180425 [linaro-7.3-2018.05 revision d29120a424ecfbc167ef90065c0eeb7f91977701] (Linaro GCC 7.3-2018.05) ) #1 SMP PREEMPT Sun Jun 18 04:26:14 UTC 2023

Linux version 4.9.337-tegra (buildbrain@mobile-u64-5499-d7000) (gcc version 7.3.1 20180425 [linaro-7.3-2018.05 revision d29120a424ecfbc167ef90065c0eeb7f91977701] (Linaro GCC 7.3-2018.05) ) #1 SMP PREEMPT Mon Nov 4 23:41:41 PST 2024


In [21]:
cat /proc/version  # after build
# => Linux version 4.9.299-tegra (jetson@jetson-nano) (gcc version 7.5.0 (Ubuntu/Linaro 7.5.0-3ubuntu1~18.04) ) #1 SMP PREEMPT Sat Jun 17 16:52:07 JST 2023

Linux version 4.9.337 (jetson@jetson-nano) (gcc version 7.5.0 (Ubuntu/Linaro 7.5.0-3ubuntu1~18.04) ) #1 SMP PREEMPT Sat May 3 19:59:09 JST 2025


In [4]:
uname -a
# => Linux jetson-nano 4.9.299-tegra #1 SMP PREEMPT Tue Nov 22 09:24:39 PST 2022 aarch64 aarch64 aarch64 GNU/Linux
# => Linux jetson-nano 4.9.299-tegra #1 SMP PREEMPT Sat Jun 17 16:52:07 JST 2023 aarch64 aarch64 aarch64 GNU/Linux
# => Linux jetson-nano 4.9.299-tegra #1 SMP PREEMPT Sun Jun 18 04:26:14 UTC 2023 aarch64 aarch64 aarch64 GNU/Linux

Linux jetson-nano 4.9.337-tegra #1 SMP PREEMPT Mon Nov 4 23:41:41 PST 2024 aarch64 aarch64 aarch64 GNU/Linux


In [23]:
uname -a  # after build

Linux jetson-nano 4.9.337 #1 SMP PREEMPT Sat May 3 19:59:09 JST 2025 aarch64 aarch64 aarch64 GNU/Linux


#### Workspace settings

In [1]:
#PUBLIC_SOURCES_URL=https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.1/Sources/T210/public_sources.tbz2   # 32.7.1
#PUBLIC_SOURCES_URL=https://developer.nvidia.com/downloads/remack-sdksjetpack-463r32releasev73sourcest210publicsourcestbz2 # 32.7.3
PUBLIC_SOURCES_URL=https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.6/sources/T210/public_sources.tbz2    # 32.7.6

In [2]:
#BASEDIR=/
#BASEDIR=.
#BASEDIR=~
#BASEDIR=/workspaces/jetson_nano_kvm
BASEDIR=~/jetson_nano_kvm
L4T_ROOT=${BASEDIR%*/}/Linux_for_Tegra
SOURCE_PUBLIC=$L4T_ROOT/source/public
KERNEL_49=$SOURCE_PUBLIC/kernel/kernel-4.9
N=$(( $(nproc) + 1 ))

In [3]:
# 1.Set the shell variable with the command:
export TEGRA_KERNEL_OUT=${SOURCE_PUBLIC:?}/build
# Where:
#   <outdir> is the desired destination for the compiled kernel.
export KERNEL_MODULES_OUT=${SOURCE_PUBLIC:?}/modules

In [4]:
#INSTALL_MOD_PATH=/
#INSTALL_MOD_PATH=$BASEDIR/rootfs/
INSTALL_MOD_PATH=$KERNEL_MODULES_OUT

In [5]:
cat <<EOF
PUBLIC_SOURCES_URL : $PUBLIC_SOURCES_URL
BASEDIR            : $BASEDIR
L4T_ROOT           : $L4T_ROOT
SOURCE_PUBLIC      : $SOURCE_PUBLIC
KERNEL_49          : $KERNEL_49
TEGRA_KERNEL_OUT   : $TEGRA_KERNEL_OUT
KERNEL_MODULES_OUT : $KERNEL_MODULES_OUT
INSTALL_MOD_PATH   : $INSTALL_MOD_PATH
CROSS_COMPILE      : $CROSS_COMPILE
LOCALVERSION       : $LOCALVERSION
N                  : $N
EOF

PUBLIC_SOURCES_URL : https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.6/sources/T210/public_sources.tbz2
BASEDIR            : /home/jetson/jetson_nano_kvm
L4T_ROOT           : /home/jetson/jetson_nano_kvm/Linux_for_Tegra
SOURCE_PUBLIC      : /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public
KERNEL_49          : /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9
TEGRA_KERNEL_OUT   : /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build
KERNEL_MODULES_OUT : /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/modules
INSTALL_MOD_PATH   : /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/modules
CROSS_COMPILE      : 
LOCALVERSION       : 
N                  : 5


#### Installs dependencies for getting/building the kernel

In [6]:
sudo -S pwd <~/.mypassword

[sudo] password for jetson: /home/jetson/jetson_nano_kvm


In [15]:
#sudo apt update && sudo apt-get install -y build-essential bc git curl wget xxd kmod libssl-dev
sudo apt-get update && sudo apt-get -y upgrade && sudo apt-get -y install --no-install-recommends \
    git-core \
    build-essential bc \
    libncurses5 libncurses5-dev sudo vim kmod usbutils \
    git curl wget xxd \
    ca-certificates git-lfs

Get:1 file:/var/cuda-repo-l4t-10-2-local  InRelease
Ign:1 file:/var/cuda-repo-l4t-10-2-local  InRelease
Get:2 file:/var/visionworks-repo  InRelease
Ign:2 file:/var/visionworks-repo  InRelease
Get:3 file:/var/visionworks-sfm-repo  InRelease
Ign:3 file:/var/visionworks-sfm-repo  InRelease
Get:4 file:/var/visionworks-tracking-repo  InRelease
Ign:4 file:/var/visionworks-tracking-repo  InRelease
Get:5 file:/var/cuda-repo-l4t-10-2-local  Release [564 B]
Get:6 file:/var/visionworks-repo  Release [2,001 B]
Get:5 file:/var/cuda-repo-l4t-10-2-local  Release [564 B]
Get:7 file:/var/visionworks-sfm-repo  Release [2,005 B]             
Get:6 file:/var/visionworks-repo  Release [2,001 B]
Get:7 file:/var/visionworks-sfm-repo  Release [2,005 B]                        
Get:8 file:/var/visionworks-tracking-repo  Release [2,010 B]                   
Get:8 file:/var/visionworks-tracking-repo  Release [2,010 B]                   
Hit:9 https://repo.download.nvidia.com/jetson/common r32.7 InRelease         

#### Cleanup

In [22]:
rm -rf "${SOURCE_PUBLIC:?}"
rm -rf "${BASEDIR:?}"/l4t-gcc
rm -v "${BASEDIR:?}"/public_sources.tbz2
rm -v "${BASEDIR:?}"/kernel*.tar.gz || true

rm: cannot remove '/home/jetson/jetson_nano_kvm/public_sources.tbz2': No such file or directory
rm: cannot remove '/home/jetson/jetson_nano_kvm/kernel*.tar.gz': No such file or directory


#### Linux kernel

The linux kernel has a config file which dictates which kernel options are enabled in the compilation process. What we need to do is enable these options, which are

```sh
CONFIG_KVM=y
CONFIG_VHOST_NET=m
```

When uncompressed, the `public_sources.tbz2` file will appear at `Linux_for_Tegra`. We also need to unpack at `Linux_for_Tegra/source/public/kernel_src.tbz2`.
The config file for tegra is at `Linux_for_Tegra/source/public/kernel/kernel-4.9/arch/arm64/configs/tegra_defconfig`

So let's do all of this in one shot. Remember that you'd have to change the kernel version and the link if you want newer kernels, and you should pick the kernel that matches your release for better compatibility. So:

In [23]:
# Gets the kernel
wget -O "$BASEDIR"/public_sources.tbz2 $PUBLIC_SOURCES_URL

--2025-05-03 17:25:53--  https://developer.download.nvidia.com/embedded/L4T/r32_Release_v7.6/sources/T210/public_sources.tbz2
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.55.107.118, 23.55.107.116
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.55.107.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162171868 (155M) [application/octet-stream]
Saving to: ‘/home/jetson/jetson_nano_kvm/public_sources.tbz2’

/home/jetson/jetson 100%[===================>] 154.66M  88.9MB/s    in 1.7s    

2025-05-03 17:25:56 (88.9 MB/s) - ‘/home/jetson/jetson_nano_kvm/public_sources.tbz2’ saved [162171868/162171868]



In [24]:
tar -C "$BASEDIR" -xf "$BASEDIR"/public_sources.tbz2

In [25]:
ls -AF $SOURCE_PUBLIC

FreeRTOSV8.1.2_src.tbz2              gstomx1_src.tbz2.sha1sum
FreeRTOSV8.1.2_src.tbz2.sha1sum      kernel_src.tbz2
dtc-1.4.0.tar.bz2                    kernel_src.tbz2.sha1sum
dtc-1.4.0.tar.bz2.sha1sum            libgstnvdrmvideosink_src.tbz2
gst-nvarguscamera_src.tbz2           libgstnvdrmvideosink_src.tbz2.sha1sum
gst-nvarguscamera_src.tbz2.sha1sum   libgstnvvideosinks_src.tbz2
gst-nvcompositor_src.tbz2            libgstnvvideosinks_src.tbz2.sha1sum
gst-nvcompositor_src.tbz2.sha1sum    nv_public_src_build.sh*
gst-nvtee_src.tbz2                   nvgstapps_src.tbz2
gst-nvtee_src.tbz2.sha1sum           nvgstapps_src.tbz2.sha1sum
gst-nvv4l2camera_src.tbz2            nvidia-xconfig_src.tbz2
gst-nvv4l2camera_src.tbz2.sha1sum    nvidia-xconfig_src.tbz2.sha1sum
gst-nvvidconv_src.tbz2               nvsample_cudaprocess_src.tbz2
gst-nvvidconv_src.tbz2.sha1sum       nvsample_cudaprocess_src.tbz2.sha1sum
gst-nvvideo4linux2_src.tbz2          opencv_gst_samples_src.tbz2
gst-nvvideo4linux2_src.tbz

In [26]:
ls -lAF $SOURCE_PUBLIC/kernel_src.tbz2

-rw-r--r-- 1 jetson jetson 138271314 11月  5 16:45 /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel_src.tbz2


In [27]:
tar -C $SOURCE_PUBLIC -xf $SOURCE_PUBLIC/kernel_src.tbz2
# =>
# hardware/
# kernel/
# nvbuild.sh
# nvcommon_build.sh

In [28]:
ls -AF $KERNEL_49

.cocciconfig            build.config.cuttlefish.aarch64  kernel/
.get_maintainer.ignore  build.config.cuttlefish.x86_64   lib/
.gitattributes          build.config.goldfish.arm        mm/
.mailmap                build.config.goldfish.arm64      net/
COPYING                 build.config.goldfish.mips       rt-patches/
CREDITS                 build.config.goldfish.mips64     samples/
Documentation/          build.config.goldfish.x86        scripts/
Kbuild                  build.config.goldfish.x86_64     security/
Kconfig                 build.config.x86_64              signing_key
MAINTAINERS             certs/                           signing_key.x509
Makefile                crypto/                          sound/
README                  drivers/                         tools/
REPORTING-BUGS          firmware/                        usr/
arch/                   fs/                              verity_dev_keys.x509
block/                  include/                         virt/
build.co

Compiling the kernel now would already activate KVM, but we would still miss an important feature that makes virtualization much faster: the irq chip. Without it, virtualization is still possible but an emulated irq chip is much slower. On `firecracker` (a virtualization tool written by AWS), it will not work as it requires this.

What we need to do is specify, in the device tree, the features of the irq chip on the CPU. The device tree is a file that contains addresses for all devices on the Jetson Nano chip.

This must be done by hand. Apply the patch below to the file `Linux_for_Tegra/source/public/kernel_src/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-bthrot-cdev.dtsi`. Don't use the patch tool as it'll likely not work, just do it by hand:

In [29]:
grep -A5 '0x0 0x50041000 0x0 0x1000' -r $SOURCE_PUBLIC/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc || true
# => tegra210-soc-base.dtsi

/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi:		reg = <0x0 0x50041000 0x0 0x1000
/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-		       0x0 0x50042000 0x0 0x0100>;
/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-		status = "disabled";
/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-	};
/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-
/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi-	lic: interrupt-controller@60004000 {


In [30]:
# cp -a $SOURCE_PUBLIC/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi{,.orig}
sed -i.orig -e ' s/0x0 0x50042000 0x0 0x0100>;/0x0 0x50042000 0x0 0x2000\n\t\t 0x0 0x50044000 0x0 0x2000\n\t\t 0x0 0x50046000 0x0 0x2000>;\n\t\tinterrupts = <GIC_PPI 9 (GIC_CPU_MASK_SIMPLE(4) | IRQ_TYPE_LEVEL_HIGH)>;/' \
  $SOURCE_PUBLIC/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi
# Restore:
# mv $SOURCE_PUBLIC/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi{.orig,}

In [32]:
diff -u $SOURCE_PUBLIC/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi{.orig,} || true

--- /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi.orig	2024-11-05 16:35:18.000000000 +0900
+++ /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi	2025-05-03 17:31:47.920051482 +0900
@@ -351,7 +351,10 @@
 		#interrupt-cells = <3>;
 		interrupt-controller;
 		reg = <0x0 0x50041000 0x0 0x1000
-		       0x0 0x50042000 0x0 0x0100>;
+		       0x0 0x50042000 0x0 0x2000
+		 0x0 0x50044000 0x0 0x2000
+		 0x0 0x50046000 0x0 0x2000>;
+		interrupts = <GIC_PPI 9 (GIC_CPU_MASK_SIMPLE(4) | IRQ_TYPE_LEVEL_HIGH)>;
 		status = "disabled";
 	};
 



```diff
--- a/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi     2020-08-31 08:40:36.602176618 +0800
+++ b/hardware/nvidia/soc/t210/kernel-dts/tegra210-soc/tegra210-soc-base.dtsi     2020-08-31 08:41:45.223679918 +0800
@@ -351,7 +351,10 @@
                #interrupt-cells = <3>;
                interrupt-controller;
                reg = <0x0 0x50041000 0x0 0x1000
-                      0x0 0x50042000 0x0 0x0100>;
+                       0x0 0x50042000 0x0 0x2000
+                       0x0 0x50044000 0x0 0x2000
+                       0x0 0x50046000 0x0 0x2000>;
+               interrupts = <GIC_PPI 9 (GIC_CPU_MASK_SIMPLE(4) | IRQ_TYPE_LEVEL_HIGH)>;
                status = "disabled";
        };
```

as you see, we added more `reg` and `interrupts`. Now, when we compile the kernel image, we'll also compile device tree files from this `dsti` file.

[jetson nanoのカーネルをソースからビルドする (zenn.dev)](https://zenn.dev/tetsu_koba/articles/7d49c86da7a4b0)

Now we should compile everything:

In [38]:
[ ! -d "${SOURCE_PUBLIC}.orig" ] && rsync -ah --del --stats "${SOURCE_PUBLIC}/" "${SOURCE_PUBLIC}.orig"


Number of files: 68,911 (reg: 64,386, dir: 4,496, link: 29)
Number of created files: 0
Number of deleted files: 0
Number of regular files transferred: 0
Total file size: 996.51M bytes
Total transferred file size: 0 bytes
Literal data: 0 bytes
Matched data: 0 bytes
File list size: 0
File list generation time: 0.002 seconds
File list transfer time: 0.000 seconds
Total bytes sent: 1.19M
Total bytes received: 4.83K

sent 1.19M bytes  received 4.83K bytes  342.38K bytes/sec
total size is 996.51M  speedup is 831.58


In [ ]:
# Remove most generated files but keep the config and enough build support to build external modules
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" clean
# Remove all generated files + config + various backup files + remove editor backup and patch files
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" distclean

In [40]:
# Generates the config file (you should manually enable/disable some missing by pressing y/n and enter)
# ARCH が提供する defconfig ファイルのデフォルト値で新しい設定ファイルを生成します。ソースに付属していたデフォルト設定ファイルに戻すにはこのオプションを使用してください。
make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" tegra_defconfig

make: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'
make[1]: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  HOSTCC  scripts/basic/fixdep
  GEN     ./Makefile
  HOSTCC  scripts/kconfig/conf.o
  SHIPPED scripts/kconfig/zconf.tab.c
  SHIPPED scripts/kconfig/zconf.lex.c
  SHIPPED scripts/kconfig/zconf.hash.c
  HOSTCC  scripts/kconfig/zconf.tab.o
  HOSTLD  scripts/kconfig/conf
#
# configuration written to .config
#
make[1]: Leaving directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
make: Leaving directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'


In [45]:
[ ! -d "${TEGRA_KERNEL_OUT:?}"/.config.orig ] && mv "${TEGRA_KERNEL_OUT:?}"/.config{,.orig}

In [ ]:
# コンフィグをカスタマイズ (interactive)
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" nconfig

In [ ]:
#
# KVM
#

# KVM (Kernel Virtual Machine)を有効化。ホスト側仮想化の基本機能（/dev/kvmデバイス）
# | Symbol: KVM [=n]
# | Type  : boolean
# | Prompt: Kernel-based Virtual Machine (KVM) support
# |   Location:
# | (1) -> Virtualization (VIRTUALIZATION [=y])
# |   Defined at arch/arm64/kvm/Kconfig:22
# |   Depends on: VIRTUALIZATION [=y] && OF [=y]
# |   Selects: MMU_NOTIFIER [=y] && PREEMPT_NOTIFIERS [=n] && HAVE_KVM_CPU_RELAX_INTERCEPT [=n] && HAVE_KVM_ARCH_TLB_FLUSH_ALL [=n] && KVM_MMIO [=n] && KVM_ARM_HOST [=n] && KVM_GENERIC_DIRTYLOG_READ

# Vhost-Netを有効化。仮想マシンのネットワークをカーネル空間で高速化するモジュール（virtio-net用）（モジュール可）
# | Symbol: VHOST_NET [=n]
# | Type  : tristate
# | Prompt: Host kernel accelerator for virtio net
# |   Location:
# | (1) -> Virtualization (VIRTUALIZATION [=y])
# |   Defined at drivers/vhost/Kconfig:1
# |   Depends on: VIRTUALIZATION [=y] && NET [=y] && EVENTFD [=y] && (TUN [=y] || !TUN [=y]) && (MACVTAP [=m] || !MACVTAP [=m])
# |   Selects: VHOST [=n]

#
# IP set
#

# ipset機能を有効化。大量IPアドレスの集合を効率的に扱うため、kube-proxyがエンドポイントやサービスCIDRをセット管理する際に必要（モジュール可）
# | Symbol: IP_SET [=n]
# | Type  : tristate
# | Prompt: IP set support
# |   Location:
# |     -> Networking support (NET [=y])
# |       -> Networking options
# | (1)     -> Network packet filtering framework (Netfilter) (NETFILTER [=y])
# |   Defined at net/netfilter/ipset/Kconfig:1
# |   Depends on: NET [=y] && INET [=y] && NETFILTER [=y]
# |   Selects: NETFILTER_NETLINK [=m]

# iptablesでipsetを利用するためのモジュールを有効化。上記 `CONFIG_IP_SET` とセットで必要。（モジュール可）
# | Symbol: NETFILTER_XT_SET [=n]
# | Type  : tristate
# | Prompt: set target and match support
# |   Location:
# |     -> Networking support (NET [=y])
# |       -> Networking options
# |         -> Network packet filtering framework (Netfilter) (NETFILTER [=y])
# |           -> Core Netfilter Configuration
# | (1)         -> Netfilter Xtables support (required for ip_tables) (NETFILTER_XTABLES [=m])
# |   Defined at net/netfilter/Kconfig:644
# |   Depends on: NET [=y] && INET [=y] && NETFILTER [=y] && NETFILTER_XTABLES [=m] && IP_SET [=n] && NETFILTER_ADVANCED [=y]

#
# iSCSI
#

# | Symbol: BASEDIR_CORE [=n]
# | Type  : tristate
# | Prompt: Generic Target Core Mod (TCM) and ConfigFS Infrastructure
# |   Location:
# | (1) -> Device Drivers
# |   Defined at drivers/target/Kconfig:2
# |   Depends on: SCSI [=y] && BLOCK [=y]
# |   Selects: CONFIGFS_FS [=y] && CRC_T10DIF [=y]

# | Symbol: ISCSI_BASEDIR [=n]
# | Type  : tristate
# | Prompt: Linux-iSCSI.org iSCSI Target Mode Stack
# |   Location:
# |     -> Device Drivers
# | (1)   -> Generic Target Core Mod (TCM) and ConfigFS Infrastructure (BASEDIR_CORE [=n])
# |   Defined at drivers/target/iscsi/Kconfig:1
# |   Depends on: BASEDIR_CORE [=n] && NET [=y]
# |   Selects: CRYPTO [=y] && CRYPTO_CRC32C [=y] && CRYPTO_CRC32C_INTEL [=n]

# | Symbol: SCSI_ISCSI_ATTRS [=n]
# | Type  : tristate
# | Prompt: iSCSI Transport Attributes
# |   Location:
# |     -> Device Drivers
# |       -> SCSI device support
# | (1)     -> SCSI Transports
# |   Defined at drivers/scsi/Kconfig:272
# |   Depends on: SCSI [=y] && NET [=y]
# |   Selects: BLK_DEV_BSGLIB [=n]
# |   Selected by: ISCSI_TCP [=n] && SCSI_LOWLEVEL [=y] && SCSI [=y] && INET [=y] || SCSI_CXGB3_ISCSI [=n] && SCSI_LOWLEVEL [=y] && SCSI [=y] && PCI [=y] && INET [=y] && (IPV6 [=y] || IPV6 [=y]=n)

# ISCSI_TCP
# | Symbol: ISCSI_TCP [=n]
# | Type  : tristate
# | Prompt: iSCSI Initiator over TCP/IP
# |   Location:
# |     -> Device Drivers
# |       -> SCSI device support
# | (1)     -> SCSI low-level drivers (SCSI_LOWLEVEL [=y])
# |   Defined at drivers/scsi/Kconfig:307
# |   Depends on: SCSI_LOWLEVEL [=y] && SCSI [=y] && INET [=y]
# |   Selects: CRYPTO [=y] && CRYPTO_MD5 [=m] && CRYPTO_CRC32C [=y] && SCSI_ISCSI_ATTRS [=n]

# ISCSI_BOOT_SYSFS
# | Symbol: ISCSI_BOOT_SYSFS [=n]
# | Type  : tristate
# | Prompt: iSCSI Boot Sysfs Interface
# |   Location:
# |     -> Device Drivers
# |       -> SCSI device support
# | (1)     -> SCSI low-level drivers (SCSI_LOWLEVEL [=y])
# |   Defined at drivers/scsi/Kconfig:331
# |   Depends on: SCSI_LOWLEVEL [=y] && SCSI [=y]
# |   Selected by: BE2ISCSI [=n] && SCSI_LOWLEVEL [=y] && PCI [=y] && SCSI [=y] && NET [=y] || SCSI_QLA_ISCSI [=n] && SCSI_LOWLEVEL [=y] && PCI [=y] && SCSI [=y] && NET [=y] || ISCSI_IBFT [=n] && AC

#
# Others
#

# RTスケジューリングのグループ制御を有効化。リアルタイム優先度のグループ管理（必要なら）
# | Symbol: RT_GROUP_SCHED [=n]
# | Type  : boolean
# | Prompt: Group scheduling for SCHED_RR/FIFO
# |   Location:
# |     -> General setup
# |       -> Control Group support (CGROUPS [=y])
# | (1)     -> CPU controller (CGROUP_SCHED [=y])
# |   Defined at init/Kconfig:1191
# |   Depends on: CGROUPS [=y] && CGROUP_SCHED [=y]

In [ ]:
# Symbol: VSOCKETS [=n]
#   │ Type  : tristate
#   │ Prompt: Virtual Socket protocol
#   │   Location:
#   │     -> Networking support (NET [=y])
#   │ (1)   -> Networking options
#   │   Defined at net/vmw_vsock/Kconfig:5
#   │   Depends on: NET [=y]

# Symbol: VHOST_VSOCK [=n]
#   │ Type  : tristate
#   │ Prompt: vhost virtio-vsock driver
#   │   Location:
#   │ (1) -> Virtualization (VIRTUALIZATION [=y])
#   │   Defined at drivers/vhost/Kconfig:22
#   │   Depends on: VIRTUALIZATION [=y] && VSOCKETS [=n] && EVENTFD [=y]
#   │   Selects: VIRTIO_VSOCKETS_COMMON [=n] && VHOST [=n]

- `CONFIG_NAMESPACES=y`: 名前空間の基本機能を有効化。各種分離に必須
- `CONFIG_NET_NS=y`: ネットワーク名前空間を有効化。Pod毎にネットワーク空間を隔離するため
- `CONFIG_PID_NS=y`: プロセスID名前空間を有効化。コンテナ内でPIDを再利用し隔離するため
- `CONFIG_IPC_NS=y`: IPC名前空間を有効化。コンテナ間でSystem V IPC等を隔離するため
- `CONFIG_UTS_NS=y`: UTS名前空間を有効化。ホスト名等の隔離に必要
- `CONFIG_CGROUPS=y`: コントロールグループ(cgroups)機能を有効化。リソース制御の基本
- `CONFIG_CGROUP_PIDS=y`: プロセス数のcgroup制限を有効化。Pod毎のプロセス数制限に使用
- `CONFIG_CGROUP_CPUACCT=y`: CPU使用量の会計機能を有効化。K8sがCPU使用統計を取得するのに必要
- `CONFIG_CGROUP_SCHED=y`: CPUスケジューラのグループ拡張を有効化。CPUシェアやクォータ制御（CFS帯域制限）に必須
- `CONFIG_CPUSETS=y`: CPUセット機能を有効化。特定CPUコアへの割り当て（CPU Pinning）に利用
- `CONFIG_CFS_BANDWIDTH=y`: CPUのCFS帯域幅制限（CPUクォータ）を有効化。PodのCPU制限実装に必要
- `CONFIG_FAIR_GROUP_SCHED=y`: CFSのグループスケジューリングを有効化。cgroup単位の公平なCPU割当てに必要
- `CONFIG_RT_GROUP_SCHED=y`: RTスケジューリングのグループ制御を有効化。リアルタイム優先度のグループ管理（必要なら）
- `CONFIG_CGROUP_DEVICE=y`: デバイスアクセス制御を有効化。コンテナ毎のデバイス許可/禁止に必要
- `CONFIG_CGROUP_FREEZER=y`: プロセス停止のcgroup制御を有効化。Podの一時停止等に利用可能
- `CONFIG_MEMCG=y`: メモリコントローラを有効化。コンテナ毎のメモリ制限に必須
- `CONFIG_CGROUP_HUGETLB=y`: hugepagesのcgroup制御を有効化。巨大ページ使用量制限（必要なら）に対応
- `CONFIG_BLK_CGROUP=y`: ブロックIOのcgroup制御を有効化。ストレージIO帯域制限に必要
- `CONFIG_BLK_DEV_THROTTLING=y`: ブロックデバイスIOスロットルを有効化。IO制限（必要なら）に対応
- `CONFIG_CGROUP_PERF=y`: パフォーマンス計測のcgroup拡張を有効化。cgroup単位のperf統計取得に必要
- `CONFIG_NET_CLS_CGROUP=y`: ネットワーククラス分類のcgroup統合を有効化（オプション、必要に応じて）
- `CONFIG_CGROUP_NET_PRIO=y`: ネットワーク優先度制御のcgroup統合を有効化（オプション）
- `CONFIG_POSIX_MQUEUE=y`: POSIXメッセージキューを有効化。コンテナランタイムやアプリが使用する可能性があるため
- `CONFIG_KEYS=y`: キー管理機能を有効化。認証や暗号化（例: overlayFSの暗号化やCephなど使用時）に必要
- `CONFIG_USER_NS=y`: (推奨オプション) ユーザー名前空間を有効化。rootlessコンテナ等に必要（任意だが推奨）
- `CONFIG_SECCOMP=y`: (推奨オプション) Seccompを有効化。KubernetesのデフォルトSeccompプロファイルによるSyscall制限に必要
- `CONFIG_OVERLAY_FS=m`: (コンテナストレージ) OverlayFSを有効化。コンテナのレイヤードFS実現に必要（Docker/Containerdのoverlayストレージ)（モジュールとしてビルド可）
- 補足: 上記の多くはK3s内のcheck-configでも「Generally Necessary」または「Optional Features」として挙げられている項目です ￼特に`CONFIG_MEMCG（メモリcgroup）など一部はJetsonの標準カーネルでは無効の場合があり、Kubernetes動作のため有効化が必要です
またCONFIG_OVERLAY_FSはコンテナイメージのレイヤー実装に必須です。Jetson NanoのデフォルトカーネルでもDocker対応のため有効化（モジュール）されていますが、未設定の場合は有効にしてください`
- `CONFIG_VETH=y`: 仮想Ethernetデバイスを有効化。Pod用のvethペア作成に必須
- `CONFIG_BRIDGE=y`: ブリッジ機能を有効化。コンテナネットワークでブリッジを利用する際に必要
- `CONFIG_BRIDGE_NETFILTER=m`: ブリッジ上のトラフィックに対するiptablesフィルタを有効化。flannel等でブリッジ経由通信をフィルタ/NATする場合に必要（モジュール可）
- `CONFIG_NETFILTER=y`: Netfilterフレームワークを有効化。iptablesによるパケットフィルタ/NAT処理の基盤（モジュール可）
- `CONFIG_IP_NF_FILTER=m`: IPv4パケットフィルタ(iptables filterテーブル)を有効化。Kubernetesのサービスやネットワークポリシーで必要（モジュール可）
- `CONFIG_IP_NF_BASEDIR_MASQUERADE=m`: MASQUERADE（アドレス偽装）ターゲットを有効化。Podから外部へのSNATやflannelの外向きトラフィック処理に必須（モジュール可）
- `CONFIG_IP_NF_BASEDIR_REJECT=m`: REJECTターゲットを有効化。不要トラフィックの拒否に使用（例：不許可ポートのICMP拒否応答)（モジュール可）
- `CONFIG_IP_NF_NAT=m`: IPv4 NAT機能を有効化。KubernetesサービスのClusterIP⇔Pod IP変換やPod外部通信のSNATに必要（モジュール可）
- `CONFIG_NF_NAT=m`: 汎用NAT機能を有効化。上記とセットで必要（モジュール可）
- `CONFIG_NF_CONNTRACK=y`: コネクショントラッキングを有効化。iptablesの状態管理やNATに必須（自動選択される場合あり）。
- `CONFIG_NETFILTER_XT_MATCH_CONNTRACK=m`: iptables拡張マッチ: Conntrack状態をマッチするモジュールを有効化。kube-proxyのルールで使用（モジュール可）
- `CONFIG_NETFILTER_XT_MATCH_ADDRTYPE=m`: iptables拡張マッチ: アドレスタイプをマッチするモジュールを有効化。Serviceの外部/内部判定等で使用（モジュール可）
- `CONFIG_NETFILTER_XT_MATCH_COMMENT=m`: iptables拡張マッチ: コメント付与用（ルール識別用）。K3sデフォルトルールで使用（モジュール可）
- `CONFIG_NETFILTER_XT_MATCH_MULTIPORT=m`: iptables拡張マッチ: 複数ポート指定用。複数サービスポートのマッチに使用（モジュール可）
- `CONFIG_IP_SET=m`: ipset機能を有効化。大量IPアドレスの集合を効率的に扱うため、kube-proxyがエンドポイントやサービスCIDRをセット管理する際に必要（モジュール可）
- `CONFIG_NETFILTER_XT_SET=m`: iptablesでipsetを利用するためのモジュールを有効化。上記 `CONFIG_IP_SET` とセットで必要。（モジュール可）
- `CONFIG_IP_VS=m`: (IPVSモード用) IPVS (IP Virtual Server)を有効化。kube-proxyをIPVSモードで利用する場合に必要（モジュール可）
- (IPVSモード用) `CONFIG_IP_VS_PROTO_TCP/UDP`=y: IPVSでTCP/UDPを扱うプロトコルサポートを有効化
- (IPVSモード用) `CONFIG_IP_VS_RR=m`: IPVSのラウンドロビン・スケジューラを有効化（モジュール可）
- (IPVSモード用) `CONFIG_IP_VS_NFCT=y`: IPVSとNetfilterのConntrack連携を有効化
- `CONFIG_VXLAN=m`: VXLANオーバーレイネットワーク機能を有効化。flannelのVXLANバックエンドに必須（モジュール可）
- `CONFIG_INET_UDP_TUNNEL=y`: VXLANで使用するUDPトンネルサポートを有効化（`CONFIG_VXLAN選択時に自動有効化）。
- `CONFIG_XFRM=y`: (任意：暗号化ネットワーク用) IPsec等のトンネリング/暗号化に必要。flannelをIPsecモードで使用する場合に要求される ￼
- `CONFIG_INET_XFRM_MODE_TRANSPORT=y`
- `CONFIG_INET_ESP=m`: (任意：暗号化ネットワーク用) IPsec ESPプロトコルサポートを有効化。暗号化パケットの処理に必要（モジュール可）
- `CONFIG_CRYPTO_AEAD=m`: (任意：暗号化ネットワーク用) 暗号アルゴリズム。ネットワーク暗号化（IPsec/WireGuard）に必要（暗号化を使う場合有効化）
- `CONFIG_CRYPTO_GCM=m`
- `CONFIG_CRYPTO_GHASH=m`
- 補足: 上記のiptables関連オプションは、K3sのkube-proxyがiptablesモードでサービスルーティングする際に必要になります。 ￼に示されるように、Jetsonカーネルでは一部がモジュールとして提供されています。また`CONFIG_IP_SETはJetson系ではデフォルト無効の場合があり ￼、Longhorn等他機能にも影響するため必ず有効化してください。`
- VXLAN機能（`CONFIG_VXLAN）もJetson向けカーネルで有効化されていることを確認してくださいVXLANにより各ノード間でOverlayネットワークを構築し、Pod間通信が実現されます。
- `CONFIG_SCSI_ISCSI_ATTRS=y`: iSCSIイニシエータ属性を有効化。iSCSIセッション管理に必要（open`-iscsiが要求）
- `CONFIG_ISCSI_TCP=m`: iSCSI over TCPクライアント機能を有効化。LonghornノードがiSCSIターゲットに接続するため（open-iscsiで使用） ￼（モジュール可）
- `CONFIG_ISCSI_BASEDIR=m`: iSCSIターゲット(LIO)機能を有効化。LonghornでボリュームをエクスポートするノードがiSCSIターゲットとして振る舞うため ￼（モジュール可）
- LIOターゲットコアを有効化。上記`CONFIG_ISCSI_BASEDIRを利用するための共通ターゲット機能 ￼※CONFIG_ISCSI_BASEDIR有効化時に自動選択。
CONFIG_BASEDIR_CORE=y`
- `CONFIG_ISCSI_BOOT_SYSFS=y`: (任意) iSCSIブート用設定(sysfs)を有効化。通常のLonghorn動作には不要（iSCSI経由でのネットブート用途）。
- 補足: Longhornの動作には各ノードでiSCSIイニシエータが必要であり、Ubuntu環境ではopen-iscsiサービスによって管理されます。Jetson Nanoの標準カーネルではこれらiSCSI関連オプションがすべて無効になっているため ￼、Longhorn導入前にカーネル再構築で有効化が必要です。特にiscsi_tcpモジュール（`CONFIG_ISCSI_TCP）が存在しないとLonghornボリュームをアタッチできませんこれらはモジュールとしてビルドし、modprobe iscsi_tcp等で読み込む形でも動作可能です。`
- また、`CONFIG_ISCSI_BASEDIRとそれに依存するCONFIG_BASEDIR_COREを有効にすることで、Linux内蔵のiSCSIターゲット(LIO`)機能が利用可能になります。Longhornでは通常ユーザー空間でボリュームを管理しますが、将来的にカーネルのLIOターゲットを使う場合に備えて有効化しておくと良いでしょう有効化するとカーネルモジュールtarget_core_mod.koおよびiscsi_target_mod.koとして提供されます
- `CONFIG_KVM=y`: KVM (Kernel Virtual Machine)を有効化。ホスト側仮想化の基本機能（/dev/kvmデバイス） 
- `CONFIG_VHOST_NET=m`: Vhost-Netを有効化。仮想マシンのネットワークをカーネル空間で高速化するモジュール（virtio-net用） ￼（モジュール可）
- `CONFIG_TUN=m`: TUN/TAPデバイスを有効化。仮想マシンとホスト間を接続する仮想ネットワークインタフェース(tapデバイス)に必須。（モジュール可）
- `CONFIG_VIRTIO=y`: Virtio仮想デバイスフレームワークを有効化。以下の各種virtioデバイスサポートに必要。
- `CONFIG_VIRTIO_NET=m`: Virtioネットワークデバイスを有効化。仮想マシン用の高性能ネットワークインタフェース（Windows VMでも対応ドライバあり）。（モジュール可）
- `CONFIG_VIRTIO_BLK=m`: Virtioブロックデバイスを有効化。仮想マシンのディスクを効率よく扱うため（Windows VMでも対応ドライバあり）。（モジュール可）
- 補足: Jetson Nanoの標準カーネルではKVMサポートが無効化されており、そのままでは/dev/kvmが存在せず仮想マシンを起動できません ￼そのため上記の`CONFIG_KVMを有効にしてカーネルを再構築する必要があります。幸い、Jetson NanoのCPU（ARM` Cortex-A57）は仮想化拡張に対応しており、再構築によりKVMを利用可能ですKVM有効化時にはあわせて`CONFIG_VHOST_NETも有効化することが推奨されています。これは仮想NICのパケット処理をユーザ空間のQEMUではなくカーネルで行い、高速化するためのモジュールです ￼`
- また、Windows仮想マシンを運用する際にはvirtioデバイス（ネットワークやブロック）が使えると性能が向上します。上記のように`CONFIG_VIRTIO_NETやCONFIG_VIRTIO_BLKを有効にしておけば、Windows用のvirtioドライバを導入することで効率的なI/Oが可能になります。`
- なお、仮想マシンのネットワークブリッジには/dev/tunデバイス（Tun/Tapドライバ）が必要になるため、`CONFIG_TUNもモジュールで構いませんので有効化してください。

In [50]:
ls -la "${TEGRA_KERNEL_OUT:?}"/.config*

-rw-rw-r-- 1 jetson jetson 168686  5月  3 18:05 /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build/.config
-rw-rw-r-- 1 jetson jetson 167605  5月  3 17:49 /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build/.config.old
-rw-rw-r-- 1 jetson jetson 167605  5月  3 17:43 /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build/.config.orig


In [49]:
# Save template
# cp "${TEGRA_KERNEL_OUT:?}"/.config ${BASEDIR:?}/Kconfig/menuconfig-kvm
# cp "${TEGRA_KERNEL_OUT:?}"/.config ${BASEDIR:?}/Kconfig/menuconfig-kvm_ipset
# cp "${TEGRA_KERNEL_OUT:?}"/.config ${BASEDIR:?}/Kconfig/menuconfig-kvm_ipset_iscsi
# cp "${TEGRA_KERNEL_OUT:?}"/.config ${BASEDIR:?}/Kconfig/menuconfig-kvm_ipset_iscsi_others

In [51]:
ls -lA ${BASEDIR:?}/Kconfig

total 672
-rw-r--r-- 1 jetson jetson    499  5月  3 05:08 fragment.conf
-rw-r--r-- 1 jetson jetson 169061  5月  3 01:39 menuconfig
-rw-rw-r-- 1 jetson jetson 168686  5月  3 18:07 menuconfig-kvm_ipset
-rw-r--r-- 1 jetson jetson 168029  5月  3 04:51 menuconfig-minimal
-rw-r--r-- 1 jetson jetson 167605  5月  3 01:39 tegra_defconfig


In [ ]:
# Restore template
cp ${BASEDIR:?}/Kconfig/menuconfig-kvm_ipset "${TEGRA_KERNEL_OUT:?}"/.config

In [53]:
diff -u "${TEGRA_KERNEL_OUT:?}"/.config{.orig,.old}

In [55]:
diff -u "${TEGRA_KERNEL_OUT:?}"/.config{.old,} || true

--- /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build/.config.old	2025-05-03 17:49:01.572101249 +0900
+++ /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build/.config	2025-05-03 18:05:35.171323684 +0900
@@ -343,6 +343,7 @@
 CONFIG_DEFAULT_CFQ=y
 # CONFIG_DEFAULT_NOOP is not set
 CONFIG_DEFAULT_IOSCHED="cfq"
+CONFIG_PREEMPT_NOTIFIERS=y
 # CONFIG_DIAG_KERNEL is not set
 CONFIG_UNINLINE_SPIN_UNLOCK=y
 CONFIG_ARCH_SUPPORTS_ATOMIC_RMW=y
@@ -463,6 +464,7 @@
 CONFIG_ARM64_ERRATUM_824069=y
 CONFIG_ARM64_ERRATUM_819472=y
 CONFIG_ARM64_ERRATUM_832075=y
+CONFIG_ARM64_ERRATUM_834220=y
 CONFIG_ARM64_ERRATUM_845719=y
 # CONFIG_ARM64_ERRATUM_843419 is not set
 CONFIG_ARM64_ERRATUM_1024718=y
@@ -841,6 +843,7 @@
 #
 CONFIG_NETFILTER_XT_MARK=m
 CONFIG_NETFILTER_XT_CONNMARK=m
+CONFIG_NETFILTER_XT_SET=m
 
 #
 # Xtables targets
@@ -921,7 +924,23 @@
 # CONFIG_NETFILTER_XT_MATCH_TCPMSS is not set
 CONFIG_NETFILTER_XT_MATCH_TIME=m
 CONFIG_NETFILTER_XT_MATCH_U32=m
-# CONFIG_IP_SET is

In [ ]:
# https://qiita.com/ysakashita/items/566e082a5d060eef5046#%E3%82%AB%E3%83%BC%E3%83%8D%E3%83%AB%E3%81%AE%E3%82%B3%E3%83%B3%E3%83%91%E3%82%A4%E3%83%AB
#REGEX='CONFIG_(KVM|VHOST_NET|VHOST_VSOCK|VSOCKETS|MACVTAP|VIRTIO|CFQ_GROUP_IOSCHED|NETFILTER_XT_SET|IP_SET.*|NET_EMATCH_IPSET|SCSI_CONSTANTS|SCSI_LOGGING|SCSI_SCAN_ASYNC|SCSI_ISCSI_ATTRS|DM_BUFIO|DM_BIO_PRISON|DM_PERSISTENT_DATA|DM_THIN_PROVISIONING|IP_NF_BASEDIR_REDIRECT|CGROUP_HUGETLB|ISCSI)='
#REGEX='CONFIG_(NAMESPACES|NET_NS|PID_NS|IPC_NS|UTS_NS|CGROUPS|CGROUP_PIDS|CGROUP_CPUACCT|CGROUP_SCHED|CPUSETS|CFS_BANDWIDTH|FAIR_GROUP_SCHED|RT_GROUP_SCHED|CGROUP_DEVICE|CGROUP_FREEZER|MEMCG|CGROUP_HUGETLB|BLK_CGROUP|BLK_DEV_THROTTLING|CGROUP_PERF|NET_CLS_CGROUP|CGROUP_NET_PRIO|POSIX_MQUEUE|KEYS|USER_NS|SECCOMP|OVERLAY_FS|VETH|BRIDGE|BRIDGE_NETFILTER|NETFILTER|IP_NF_FILTER|IP_NF_TARGET_MASQUERADE|IP_NF_TARGET_REJECT|IP_NF_NAT|NF_NAT|NF_CONNTRACK|NETFILTER_XT_MATCH_CONNTRACK|NETFILTER_XT_MATCH_ADDRTYPE|NETFILTER_XT_MATCH_COMMENT|NETFILTER_XT_MATCH_MULTIPORT|IP_SET|NETFILTER_XT_SET|IP_VS|IP_VS_PROTO_TCP|IP_VS_PROTO_UDP|IP_VS_RR|IP_VS_NFCT|VXLAN|INET_UDP_TUNNEL|XFRM|INET_XFRM_MODE_TRANSPORT|INET_ESP|CRYPTO_AEAD|CRYPTO_GCM|CRYPTO_GHASH|SCSI_ISCSI_ATTRS|ISCSI_TCP|ISCSI_TARGET|TARGET_CORE|ISCSI_BOOT_SYSFS|KVM|KVM_ARM_HOST|VHOST_NET|TUN|VIRTIO|VIRTIO_NET|VIRTIO_BLK)='
REGEX='CONFIG_(NAMESPACES|NET_NS|PID_NS|IPC_NS|UTS_NS|CGROUPS|CGROUP_PIDS|CGROUP_CPUACCT|CGROUP_DEVICE|CGROUP_FREEZER|CGROUP_SCHED|CPUSETS|MEMCG|SECCOMP|KEYS|VETH|BRIDGE|BRIDGE_NETFILTER|IP_NF_FILTER|IP_NF_TARGET_MASQUERADE|IP_NF_TARGET_REJECT|NETFILTER_XT_MATCH_ADDRTYPE|NETFILTER_XT_MATCH_CONNTRACK|NETFILTER_XT_MATCH_IPVS|NETFILTER_XT_MATCH_COMMENT|NETFILTER_XT_MATCH_MULTIPORT|NETFILTER_XT_MATCH_STATISTIC|IP_NF_NAT|NF_NAT|POSIX_MQUEUE|USER_NS|BLK_CGROUP|BLK_DEV_THROTTLING|CGROUP_PERF|CGROUP_HUGETLB|NET_CLS_CGROUP|CGROUP_NET_PRIO|CFS_BANDWIDTH|FAIR_GROUP_SCHED|RT_GROUP_SCHED|IP_NF_TARGET_REDIRECT|IP_SET|IP_VS|IP_VS_NFCT|IP_VS_PROTO_TCP|IP_VS_PROTO_UDP|IP_VS_RR|EXT4_FS|EXT4_FS_POSIX_ACL|EXT4_FS_SECURITY|VXLAN|CRYPTO|CRYPTO_AEAD|CRYPTO_GCM|CRYPTO_SEQIV|CRYPTO_GHASH|XFRM|XFRM_USER|XFRM_ALGO|INET_ESP|INET_XFRM_MODE_TRANSPORT|OVERLAY_FS)='  # from k3s check-config

In [57]:
grep -E "$REGEX" $KERNEL_49/arch/arm64/configs/tegra_defconfig
grep -E "$REGEX" $KERNEL_49/arch/arm64/configs/tegra_defconfig | wc -l # => 43

CONFIG_POSIX_MQUEUE=y
CONFIG_CGROUPS=y
CONFIG_CGROUP_FREEZER=y
CONFIG_CGROUP_PIDS=y
CONFIG_CGROUP_DEVICE=y
CONFIG_CPUSETS=y
CONFIG_CGROUP_CPUACCT=y
CONFIG_MEMCG=y
CONFIG_BLK_CGROUP=y
CONFIG_CGROUP_SCHED=y
CONFIG_CFS_BANDWIDTH=y
CONFIG_CGROUP_HUGETLB=y
CONFIG_CGROUP_PERF=y
CONFIG_NAMESPACES=y
CONFIG_USER_NS=y
CONFIG_BLK_DEV_THROTTLING=y
CONFIG_SECCOMP=y
CONFIG_INET_ESP=m
CONFIG_NETFILTER=y
CONFIG_NF_CONNTRACK=m
CONFIG_NETFILTER_XT_MATCH_ADDRTYPE=m
CONFIG_NETFILTER_XT_MATCH_COMMENT=m
CONFIG_NETFILTER_XT_MATCH_CONNTRACK=m
CONFIG_NETFILTER_XT_MATCH_MULTIPORT=m
CONFIG_IP_VS=m
CONFIG_IP_VS_PROTO_TCP=y
CONFIG_IP_VS_PROTO_UDP=y
CONFIG_IP_VS_RR=m
CONFIG_IP_VS_NFCT=y
CONFIG_IP_NF_FILTER=m
CONFIG_IP_NF_TARGET_REJECT=m
CONFIG_IP_NF_NAT=m
CONFIG_IP_NF_TARGET_MASQUERADE=m
CONFIG_BRIDGE=y
CONFIG_NET_CLS_CGROUP=y
CONFIG_CGROUP_NET_PRIO=y
CONFIG_VIRTIO_BLK=y
CONFIG_VXLAN=y
CONFIG_TUN=y
CONFIG_VETH=m
CONFIG_VIRTIO_NET=y
CONFIG_OVERLAY_FS=m
CONFIG_CRYPTO_GCM=y
43


In [58]:
zcat /proc/config.gz | grep -E "$REGEX"
zcat /proc/config.gz | grep -E "$REGEX" | wc -l # => 56

CONFIG_POSIX_MQUEUE=y
CONFIG_CGROUPS=y
CONFIG_CGROUP_FREEZER=y
CONFIG_CGROUP_PIDS=y
CONFIG_CGROUP_DEVICE=y
CONFIG_CPUSETS=y
CONFIG_CGROUP_CPUACCT=y
CONFIG_MEMCG=y
CONFIG_BLK_CGROUP=y
CONFIG_CGROUP_SCHED=y
CONFIG_FAIR_GROUP_SCHED=y
CONFIG_CFS_BANDWIDTH=y
CONFIG_CGROUP_HUGETLB=y
CONFIG_CGROUP_PERF=y
CONFIG_NAMESPACES=y
CONFIG_UTS_NS=y
CONFIG_IPC_NS=y
CONFIG_USER_NS=y
CONFIG_PID_NS=y
CONFIG_NET_NS=y
CONFIG_BLK_DEV_THROTTLING=y
CONFIG_SECCOMP=y
CONFIG_XFRM=y
CONFIG_INET_ESP=m
CONFIG_INET_XFRM_MODE_TRANSPORT=y
CONFIG_NETFILTER=y
CONFIG_BRIDGE_NETFILTER=m
CONFIG_NF_CONNTRACK=m
CONFIG_NF_NAT=m
CONFIG_NETFILTER_XT_MATCH_ADDRTYPE=m
CONFIG_NETFILTER_XT_MATCH_COMMENT=m
CONFIG_NETFILTER_XT_MATCH_CONNTRACK=m
CONFIG_NETFILTER_XT_MATCH_MULTIPORT=m
CONFIG_IP_VS=m
CONFIG_IP_VS_PROTO_TCP=y
CONFIG_IP_VS_PROTO_UDP=y
CONFIG_IP_VS_RR=m
CONFIG_IP_VS_NFCT=y
CONFIG_IP_NF_FILTER=m
CONFIG_IP_NF_TARGET_REJECT=m
CONFIG_IP_NF_NAT=m
CONFIG_IP_NF_TARGET_MASQUERADE=m
CONFIG_BRIDGE=y
CONFIG_NET_CLS_CGROUP=y
CONFIG_CGRO

In [59]:
grep -E "$REGEX" "${TEGRA_KERNEL_OUT:?}"/.config
grep -E "$REGEX" "${TEGRA_KERNEL_OUT:?}"/.config | wc -l # => ? or 61 or 67, etc.

CONFIG_POSIX_MQUEUE=y
CONFIG_CGROUPS=y
CONFIG_CGROUP_FREEZER=y
CONFIG_CGROUP_PIDS=y
CONFIG_CGROUP_DEVICE=y
CONFIG_CPUSETS=y
CONFIG_CGROUP_CPUACCT=y
CONFIG_MEMCG=y
CONFIG_BLK_CGROUP=y
CONFIG_CGROUP_SCHED=y
CONFIG_FAIR_GROUP_SCHED=y
CONFIG_CFS_BANDWIDTH=y
CONFIG_CGROUP_HUGETLB=y
CONFIG_CGROUP_PERF=y
CONFIG_NAMESPACES=y
CONFIG_UTS_NS=y
CONFIG_IPC_NS=y
CONFIG_USER_NS=y
CONFIG_PID_NS=y
CONFIG_NET_NS=y
CONFIG_BLK_DEV_THROTTLING=y
CONFIG_SECCOMP=y
CONFIG_XFRM=y
CONFIG_INET_ESP=m
CONFIG_INET_XFRM_MODE_TRANSPORT=y
CONFIG_NETFILTER=y
CONFIG_BRIDGE_NETFILTER=m
CONFIG_NF_CONNTRACK=m
CONFIG_NF_NAT=m
CONFIG_NETFILTER_XT_SET=m
CONFIG_NETFILTER_XT_MATCH_ADDRTYPE=m
CONFIG_NETFILTER_XT_MATCH_COMMENT=m
CONFIG_NETFILTER_XT_MATCH_CONNTRACK=m
CONFIG_NETFILTER_XT_MATCH_MULTIPORT=m
CONFIG_IP_SET=m
CONFIG_IP_VS=m
CONFIG_IP_VS_PROTO_TCP=y
CONFIG_IP_VS_PROTO_UDP=y
CONFIG_IP_VS_RR=m
CONFIG_IP_VS_NFCT=y
CONFIG_IP_NF_FILTER=m
CONFIG_IP_NF_TARGET_REJECT=m
CONFIG_IP_NF_NAT=m
CONFIG_IP_NF_TARGET_MASQUERADE=m
CONFIG_BR

In [60]:
#make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" oldconfig     # 依存関係を修正 (修正値は対話的に聞かれる)
make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" olddefconfig  # 依存関係を自動修正

make: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'
make[1]: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  GEN     ./Makefile
scripts/kconfig/conf  --olddefconfig Kconfig
#
# configuration written to .config
#
make[1]: Leaving directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
make: Leaving directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'


In [63]:
diff -u "${TEGRA_KERNEL_OUT:?}"/.config{.old,}

In [70]:
make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" help

make: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'
make[1]: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
Cleaning targets:
  clean		  - Remove most generated files but keep the config and
                    enough build support to build external modules
  mrproper	  - Remove all generated files + config + various backup files
  distclean	  - mrproper + remove editor backup and patch files

Configuration targets:
  config	  - Update current config utilising a line-oriented program
  nconfig         - Update current config utilising a ncurses menu based
                    program
  menuconfig	  - Update current config utilising a menu based program
  xconfig	  - Update current config utilising a Qt based front-end
  gconfig	  - Update current config utilising a GTK+ based front-end
  oldconfig	  - Update current config utilising a provided .config as base
  localmodconfig  - Update current con

In [68]:
#N=$(nproc)
N=$(( $(nproc) + 1 ))
echo $N

5


In [ ]:
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N prepare
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N modules_prepare

In [71]:
# Build the kernel
make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N

make: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'
make[1]: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  CHK     include/config/kernel.release
  GEN     ./Makefile
  UPD     include/config/kernel.release
  WRAP    arch/arm64/include/generated/asm/bugs.h
  WRAP    arch/arm64/include/generated/asm/clkdev.h
  WRAP    arch/arm64/include/generated/asm/cputime.h
  WRAP    arch/arm64/include/generated/asm/delay.h
  WRAP    arch/arm64/include/generated/asm/dma-contiguous.h
  WRAP    arch/arm64/include/generated/asm/div64.h
  WRAP    arch/arm64/include/generated/asm/early_ioremap.h
  WRAP    arch/arm64/include/generated/asm/errno.h
  CHK     include/generated/uapi/linux/version.h
  WRAP    arch/arm64/include/generated/asm/emergency-restart.h
  WRAP    arch/arm64/include/generated/asm/hw_irq.h
  WRAP    arch/arm64/include/generated/asm/ioctls.h
  WRAP    arch/arm64/include/generated/asm/ioctl.h
  WRAP

In [ ]:
# Generates the Image that we're gonna place on /boot/Image (Skip this step: it's already part of `make all`)
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target zImage
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target Image.gz

In [ ]:
# Generates the drivers. This is needed because the old driver will not work with our new Image
#   (Skip this step: it's already part of `make all`)
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target modules

In [ ]:
# Generates our modified device file trees (Skip this step: it's already part of `make all`)
# make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" -j$N --output-sync=target dtbs

In [72]:
# Installs the modules on the build folder ~/Linux_for_Tegra/source/public/build
make -C "${KERNEL_49:?}" ARCH=arm64 O="${TEGRA_KERNEL_OUT:?}" INSTALL_MOD_PATH="${KERNEL_MODULES_OUT:?}" modules_install

make: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/kernel/kernel-4.9'
make[1]: Entering directory '/home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build'
  INSTALL crypto/authenc.ko
  INSTALL crypto/authencesn.ko
  INSTALL crypto/deflate.ko
  INSTALL crypto/des_generic.ko
  INSTALL crypto/md4.ko
  INSTALL crypto/md5.ko
  INSTALL crypto/michael_mic.ko
  INSTALL crypto/tcrypt.ko
  INSTALL crypto/xor.ko
  INSTALL drivers/bcma/bcma.ko
  INSTALL drivers/block/nbd.ko
  INSTALL drivers/block/zram/zram.ko
  INSTALL drivers/bluetooth/ath3k.ko
  INSTALL drivers/bluetooth/bcm203x.ko
  INSTALL drivers/bluetooth/bfusb.ko
  INSTALL drivers/bluetooth/bpa10x.ko
  INSTALL drivers/bluetooth/btbcm.ko
  INSTALL drivers/bluetooth/btintel.ko
  INSTALL drivers/bluetooth/btmrvl.ko
  INSTALL drivers/bluetooth/btmrvl_sdio.ko
  INSTALL drivers/bluetooth/btqca.ko
  INSTALL drivers/bluetooth/btrtl.ko
  INSTALL drivers/bluetooth/btusb.ko
  INSTALL drivers/bluetooth/hci_u

Now that we have our Image, the drivers and the file trees, we should override them, but before, make a manual backup of folders we're gonna change so you can rollback if something goes wrong.

In [75]:
sudo -S pwd <~/.mypassword

[sudo] password for jetson: /home/jetson/jetson_nano_kvm


In [76]:
[ ! -d /boot.orig ] && sudo rsync -avh --del --stats /boot/ /boot.orig
[ ! -d /lib.orig ] && sudo rsync -avh --del --stats /lib/ /lib.orig

sending incremental file list
created directory /boot.orig
./
Image
Image.orig
initrd
initrd.img -> initrd.img-4.9.253-tegra
initrd.img-4.9.253-tegra
kernel_tegra210-p3448-0000-p3449-0000-b00.dtb
tegra210-jetson-tx1-p2597-2180-a01-devkit-adafruit-sph0645lm4h.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit-adafruit-uda1334a.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit-csi.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit-fe-pi-audio.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit-hdr30.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit-hdr40.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit-m2ke.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit-respeaker-4-mic-array.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit-respeaker-4-mic-lin-array.dtbo
tegra210-jetson-tx1-p2597-2180-a01-devkit.dtb
tegra210-jetson-tx1-p2597-2180-a02-devkit-24x7.dtb
tegra210-p3448-0000-p3449-0000-a00.dtb
tegra210-p3448-0000-p3449-0000-a01-adafruit-sph0645lm4h.dtbo
tegra210-p3448-0000-p3449-0000-a01-adafruit-uda1334a.dtbo
tegra2

In [ ]:
# Reset
# sudo rsync -avh --del --stats /lib.orig/ /lib
# sudo rsync -avh --del --stats /boot.orig/ /boot

Now we can `rsync` the files with the system ones (warning, untested, I used `sudo nautilus` and moved by hand on mine).

In [78]:
ls -lAF /lib/firmware | head

total 134344
-rw-r--r--  1 root root   27156  2月 18  2022 1a98-INTEL-EDK2-2-tplg.bin
drwxr-xr-x  2 root root    4096  5月  3 15:45 3com/
-rw-r--r--  1 root root   15664  2月 18  2022 NPE-B
-rw-r--r--  1 root root   15664  2月 18  2022 NPE-C
-rw-r--r--  1 root root    1684  9月 17  2021 README
drwxr-xr-x  2 root root    4096 12月 26 20:55 RTL8192E/
-rw-r--r--  1 root root    5712  2月 18  2022 WHENCE.ubuntu
lrwxrwxrwx  1 root root      16  2月 18  2022 a300_pfp.fw -> qcom/a300_pfp.fw
lrwxrwxrwx  1 root root      16  2月 18  2022 a300_pm4.fw -> qcom/a300_pm4.fw


In [79]:
ls -lAF "${KERNEL_MODULES_OUT:?}"/lib/firmware | head

total 44
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 3com/
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 acenic/
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 bnx2/
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 bnx2x/
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 cpia2/
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 cxgb3/
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 e100/
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 emi26/
drwxrwxr-x 2 jetson jetson 4096  5月  3 20:11 emi62/


In [80]:
rsync -n -rltDv --stats "${KERNEL_MODULES_OUT:?}"/lib/firmware/ /lib/firmware

sending incremental file list
./
3com/
3com/typhoon.bin
acenic/
acenic/tg1.bin
acenic/tg2.bin
bnx2/
bnx2/bnx2-mips-06-6.2.1.fw
bnx2/bnx2-mips-09-6.2.1a.fw
bnx2/bnx2-rv2p-06-6.0.15.fw
bnx2/bnx2-rv2p-09-6.0.17.fw
bnx2/bnx2-rv2p-09ax-6.0.17.fw
bnx2x/
bnx2x/bnx2x-e1-6.2.9.0.fw
bnx2x/bnx2x-e1h-6.2.9.0.fw
bnx2x/bnx2x-e2-6.2.9.0.fw
cpia2/
cpia2/stv0672_vp4.bin
cxgb3/
cxgb3/ael2005_opt_edc.bin
cxgb3/ael2005_twx_edc.bin
cxgb3/ael2020_twx_edc.bin
cxgb3/t3b_psram-1.1.0.bin
cxgb3/t3c_psram-1.1.0.bin
e100/
e100/d101m_ucode.bin
e100/d101s_ucode.bin
e100/d102e_ucode.bin
emi26/
emi26/bitstream.fw
emi26/firmware.fw
emi26/loader.fw
emi62/
emi62/bitstream.fw
emi62/loader.fw
emi62/midi.fw
emi62/spdif.fw
kaweth/
kaweth/new_code.bin
kaweth/new_code_fix.bin
kaweth/trigger_code.bin
kaweth/trigger_code_fix.bin
yamaha/
yamaha/ds1_ctrl.fw
yamaha/ds1_dsp.fw
yamaha/ds1e_ctrl.fw

Number of files: 46 (reg: 34, dir: 12)
Number of created files: 0
Number of deleted files: 0
Number of regular files transferred: 34
Tota

In [81]:
#sudo cp -r "${KERNEL_MODULES_OUT:?}"/lib/firmware/ /lib/firmware
sudo rsync -rltDv --stats "${KERNEL_MODULES_OUT:?}"/lib/firmware/ /lib/firmware

sending incremental file list
./
3com/
3com/typhoon.bin
acenic/
acenic/tg1.bin
acenic/tg2.bin
bnx2/
bnx2/bnx2-mips-06-6.2.1.fw
bnx2/bnx2-mips-09-6.2.1a.fw
bnx2/bnx2-rv2p-06-6.0.15.fw
bnx2/bnx2-rv2p-09-6.0.17.fw
bnx2/bnx2-rv2p-09ax-6.0.17.fw
bnx2x/
bnx2x/bnx2x-e1-6.2.9.0.fw
bnx2x/bnx2x-e1h-6.2.9.0.fw
bnx2x/bnx2x-e2-6.2.9.0.fw
cpia2/
cpia2/stv0672_vp4.bin
cxgb3/
cxgb3/ael2005_opt_edc.bin
cxgb3/ael2005_twx_edc.bin
cxgb3/ael2020_twx_edc.bin
cxgb3/t3b_psram-1.1.0.bin
cxgb3/t3c_psram-1.1.0.bin
e100/
e100/d101m_ucode.bin
e100/d101s_ucode.bin
e100/d102e_ucode.bin
emi26/
emi26/bitstream.fw
emi26/firmware.fw
emi26/loader.fw
emi62/
emi62/bitstream.fw
emi62/loader.fw
emi62/midi.fw
emi62/spdif.fw
kaweth/
kaweth/new_code.bin
kaweth/new_code_fix.bin
kaweth/trigger_code.bin
kaweth/trigger_code_fix.bin
yamaha/
yamaha/ds1_ctrl.fw
yamaha/ds1_dsp.fw
yamaha/ds1e_ctrl.fw

Number of files: 46 (reg: 34, dir: 12)
Number of created files: 0
Number of deleted files: 0
Number of regular files transferred: 34
Tota

In [86]:
ls -lAFd /lib/modules/4.9.*
ls -lAF /lib/modules/4.9.*

drwxr-xr-x 3 root root 4096  5月  3 15:45 /lib/modules/4.9.337/
drwxr-xr-x 3 root root 4096 12月 26 21:04 /lib/modules/4.9.337-tegra/
/lib/modules/4.9.337:
total 1240
lrwxrwxrwx 1 root root     64  5月  3 15:45 build -> /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build/
drwxr-xr-x 8 root root   4096  5月  3 15:45 kernel/
-rw-r--r-- 1 root root 386524  5月  3 15:45 modules.alias
-rw-r--r-- 1 root root 383347  5月  3 15:45 modules.alias.bin
-rw-r--r-- 1 root root  22103  5月  3 15:45 modules.builtin
-rw-r--r-- 1 root root  24723  5月  3 15:45 modules.builtin.bin
-rw-r--r-- 1 root root  52455  5月  3 15:45 modules.dep
-rw-r--r-- 1 root root  83350  5月  3 15:45 modules.dep.bin
-rw-r--r-- 1 root root    166  5月  3 15:45 modules.devname
-rw-r--r-- 1 root root  27832  5月  3 15:45 modules.order
-rw-r--r-- 1 root root     85  5月  3 15:45 modules.softdep
-rw-r--r-- 1 root root 110673  5月  3 15:45 modules.symbols
-rw-r--r-- 1 root root 135770  5月  3 15:45 modules.symbols.bin
lrwxrwxrwx 1 ro

In [87]:
ls -lAFd "${KERNEL_MODULES_OUT:?}"/lib/modules/4.9.*
ls -lAF "${KERNEL_MODULES_OUT:?}"/lib/modules/4.9.*

drwxrwxr-x 3 jetson jetson 4096  5月  3 20:11 /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/modules/lib/modules/4.9.337/
total 1240
lrwxrwxrwx 1 jetson jetson     64  5月  3 20:11 build -> /home/jetson/jetson_nano_kvm/Linux_for_Tegra/source/public/build/
drwxrwxr-x 8 jetson jetson   4096  5月  3 20:11 kernel/
-rw-r--r-- 1 jetson jetson 386524  5月  3 20:11 modules.alias
-rw-r--r-- 1 jetson jetson 383347  5月  3 20:11 modules.alias.bin
-rw-rw-r-- 1 jetson jetson  22103  5月  3 20:11 modules.builtin
-rw-r--r-- 1 jetson jetson  24723  5月  3 20:11 modules.builtin.bin
-rw-r--r-- 1 jetson jetson  52455  5月  3 20:11 modules.dep
-rw-r--r-- 1 jetson jetson  83350  5月  3 20:11 modules.dep.bin
-rw-r--r-- 1 jetson jetson    166  5月  3 20:11 modules.devname
-rw-rw-r-- 1 jetson jetson  27832  5月  3 20:11 modules.order
-rw-r--r-- 1 jetson jetson     85  5月  3 20:11 modules.softdep
-rw-r--r-- 1 jetson jetson 110673  5月  3 20:11 modules.symbols
-rw-r--r-- 1 jetson jetson 135770  5月  3 20:11 modu

In [89]:
# -a == -rlptgoD
rsync -n -rltDv --stats "${KERNEL_MODULES_OUT:?}"/lib/modules/ /lib/modules

sending incremental file list
./
4.9.337/
4.9.337/modules.alias
4.9.337/modules.alias.bin
4.9.337/modules.builtin
4.9.337/modules.builtin.bin
4.9.337/modules.dep
4.9.337/modules.dep.bin
4.9.337/modules.devname
4.9.337/modules.order
4.9.337/modules.softdep
4.9.337/modules.symbols
4.9.337/modules.symbols.bin
4.9.337/kernel/
4.9.337/kernel/crypto/
4.9.337/kernel/crypto/authenc.ko
4.9.337/kernel/crypto/authencesn.ko
4.9.337/kernel/crypto/deflate.ko
4.9.337/kernel/crypto/des_generic.ko
4.9.337/kernel/crypto/md4.ko
4.9.337/kernel/crypto/md5.ko
4.9.337/kernel/crypto/michael_mic.ko
4.9.337/kernel/crypto/tcrypt.ko
4.9.337/kernel/crypto/xor.ko
4.9.337/kernel/drivers/
4.9.337/kernel/drivers/bcma/
4.9.337/kernel/drivers/bcma/bcma.ko
4.9.337/kernel/drivers/block/
4.9.337/kernel/drivers/block/nbd.ko
4.9.337/kernel/drivers/block/zram/
4.9.337/kernel/drivers/block/zram/zram.ko
4.9.337/kernel/drivers/bluetooth/
4.9.337/kernel/drivers/bluetooth/ath3k.ko
4.9.337/kernel/drivers/bluetooth/bcm203x.ko
4.9.33

In [90]:
#sudo cp -r "${KERNEL_MODULES_OUT:?}"/lib/modules/ /lib/modules
sudo rsync -rltDv --stats "${KERNEL_MODULES_OUT:?}"/lib/modules/ /lib/modules

sending incremental file list
./
4.9.337/
4.9.337/modules.alias
4.9.337/modules.alias.bin
4.9.337/modules.builtin
4.9.337/modules.builtin.bin
4.9.337/modules.dep
4.9.337/modules.dep.bin
4.9.337/modules.devname
4.9.337/modules.order
4.9.337/modules.softdep
4.9.337/modules.symbols
4.9.337/modules.symbols.bin
4.9.337/kernel/
4.9.337/kernel/crypto/
4.9.337/kernel/crypto/authenc.ko
4.9.337/kernel/crypto/authencesn.ko
4.9.337/kernel/crypto/deflate.ko
4.9.337/kernel/crypto/des_generic.ko
4.9.337/kernel/crypto/md4.ko
4.9.337/kernel/crypto/md5.ko
4.9.337/kernel/crypto/michael_mic.ko
4.9.337/kernel/crypto/tcrypt.ko
4.9.337/kernel/crypto/xor.ko
4.9.337/kernel/drivers/
4.9.337/kernel/drivers/bcma/
4.9.337/kernel/drivers/bcma/bcma.ko
4.9.337/kernel/drivers/block/
4.9.337/kernel/drivers/block/nbd.ko
4.9.337/kernel/drivers/block/zram/
4.9.337/kernel/drivers/block/zram/zram.ko
4.9.337/kernel/drivers/bluetooth/
4.9.337/kernel/drivers/bluetooth/ath3k.ko
4.9.337/kernel/drivers/bluetooth/bcm203x.ko
4.9.33

Now we must also update the boot folder:

In [91]:
ls -lAF "${TEGRA_KERNEL_OUT:?}"/arch/arm64/boot

total 57244
-rw-rw-r-- 1 jetson jetson      124  5月  3 20:00 .Image.cmd
-rw-rw-r-- 1 jetson jetson      147  5月  3 20:00 .Image.gz.cmd
-rw-rw-r-- 1 jetson jetson      141  5月  3 20:00 .zImage.cmd
-rw-rw-r-- 1 jetson jetson 35383304  5月  3 20:00 Image
-rw-rw-r-- 1 jetson jetson 11648664  5月  3 20:00 Image.gz
drwxrwxr-x 3 jetson jetson     4096  5月  3 19:13 dts/
-rw-rw-r-- 1 jetson jetson 11648664  5月  3 20:00 zImage


In [92]:
ls -lAF /boot

total 87412
-rw-r--r-- 1 root root 34707464 11月  5 16:46 Image
-rw-r--r-- 1 root root 34707464 11月  5 16:46 Image.orig
drwxr-xr-x 2 root root     4096  2月 23  2022 dtb/
drwxr-xr-x 2 root root     4096 12月 26 21:04 extlinux/
drwxr-xr-x 2 root root     4096 12月 26 20:45 grub/
-rw-r--r-- 1 root root  7160200 11月  5 16:46 initrd
lrwxrwxrwx 1 root root       24 12月 26 20:45 initrd.img -> initrd.img-4.9.253-tegra
-rw-r--r-- 1 root root  9924184  4月 30 23:31 initrd.img-4.9.253-tegra
-rw-r--r-- 1 root root   237268  2月 23  2022 kernel_tegra210-p3448-0000-p3449-0000-b00.dtb
-rw-r--r-- 1 root root     1077 11月  5 16:46 tegra210-jetson-tx1-p2597-2180-a01-devkit-adafruit-sph0645lm4h.dtbo
-rw-r--r-- 1 root root     1073 11月  5 16:46 tegra210-jetson-tx1-p2597-2180-a01-devkit-adafruit-uda1334a.dtbo
-rw-r--r-- 1 root root     1856 11月  5 16:46 tegra210-jetson-tx1-p2597-2180-a01-devkit-csi.dtbo
-rw-r--r-- 1 root root     2831 11月  5 16:46 tegra210-jetson-tx1-p2597-2180-a01-devkit-fe-pi-audio.dtbo
-rw-r

In [93]:
rsync -nrltDv --stats "${TEGRA_KERNEL_OUT:?}"/arch/arm64/boot/ /boot

sending incremental file list
./
.Image.cmd
.Image.gz.cmd
.zImage.cmd
Image
Image.gz
zImage
dts/
dts/tegra210-jetson-cv-base-p2597-2180-a00.dtb
dts/tegra210-jetson-cv-p2597-2180-a00-auo-1080p-edp.dtb
dts/tegra210-jetson-cv-p2597-2180-a00.dtb
dts/tegra210-jetson-cv-p2597-2180-imx274-hdmi.dtb
dts/tegra210-jetson-e-base-p2595-0000-a00.dtb
dts/tegra210-jetson-e-p2595-0000-a00-00.dtb
dts/tegra210-jetson-tx1-imx274-dp-hdmi.dtb
dts/tegra210-jetson-tx1-p2597-2180-a01-android-devkit.dtb
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-adafruit-sph0645lm4h.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-adafruit-uda1334a.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-csi.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-fe-pi-audio.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-hdr30.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-hdr40.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-m2ke.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-respeaker-4-mic-array.dtbo
dts/tegra210-je

In [94]:
sudo rsync -rltDv --stats "${TEGRA_KERNEL_OUT:?}"/arch/arm64/boot/ /boot

sending incremental file list
./
.Image.cmd
.Image.gz.cmd
.zImage.cmd
Image
Image.gz
zImage
dts/
dts/tegra210-jetson-cv-base-p2597-2180-a00.dtb
dts/tegra210-jetson-cv-p2597-2180-a00-auo-1080p-edp.dtb
dts/tegra210-jetson-cv-p2597-2180-a00.dtb
dts/tegra210-jetson-cv-p2597-2180-imx274-hdmi.dtb
dts/tegra210-jetson-e-base-p2595-0000-a00.dtb
dts/tegra210-jetson-e-p2595-0000-a00-00.dtb
dts/tegra210-jetson-tx1-imx274-dp-hdmi.dtb
dts/tegra210-jetson-tx1-p2597-2180-a01-android-devkit.dtb
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-adafruit-sph0645lm4h.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-adafruit-uda1334a.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-csi.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-fe-pi-audio.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-hdr30.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-hdr40.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-m2ke.dtbo
dts/tegra210-jetson-tx1-p2597-2180-a01-devkit-respeaker-4-mic-array.dtbo
dts/tegra210-je

#### Check

In [95]:
find /lib/firmware /lib/modules /boot ! -user root

#### Cleanup

In [ ]:
# rm -v "${BASEDIR:?}"/kernel*.tar.gz

#### Archive

In [ ]:
# -a, --auto-compress
# -z, --gzip
# -h, --dereference
# -f, --file=ARCHIVE
# tar -C "${KERNEL_MODULES_OUT:?}" -acf "${BASEDIR:?}"/kernel-4.9.299-tegra-boot-lib.tar.gz -C "${TEGRA_KERNEL_OUT:?}"/arch/arm64 boot lib
# tar -C "${BASEDIR:?}" -acf "${BASEDIR:?}"/kernel-4.9.299-tegra.tar.gz "${TEGRA_KERNEL_OUT:?}" ${SOURCE_PUBLIC:?}/kernel/kernel-4.9 "${KERNEL_MODULES_OUT:?}"/lib

Notice that we copied all of the dtb files, there are many for different models, but just one that we should use. Run

In [ ]:
sudo dmesg | grep -i kernel

to discover yours. Example of mine:

```log
[    0.236710] DTS File Name: /home/lz/Linux_for_Tegra/source/public/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a00.dts
```

In [96]:
sudo dmesg | grep -i kernel | grep DTS

[    0.219485] DTS File Name: /dvs/git/dirty/git-master_linux/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a02.dts
[    0.424164] DTS File Name: /dvs/git/dirty/git-master_linux/kernel/kernel-4.9/arch/arm64/boot/dts/../../../../../../hardware/nvidia/platform/t210/porg/kernel-dts/tegra210-p3448-0000-p3449-0000-a02.dts


In [104]:
DTS=$(sudo dmesg | grep -i kernel | grep DTS | tail -1)
DTS=/boot/${DTS##*/}
DTB=${DTS%.dts}.dtb
echo $DTB

/boot/tegra210-p3448-0000-p3449-0000-a02.dtb


Wait, wtf? Why this is a local file? I don't know what's happening, but this should show you which one is being used. You're gonna need its name. The file is already at `/boot`.

You might wonder that since we replaced all the device tree files on `/boot`, then it should load the modified one already. Somehow, in my case, it didn't. I think it has to do with the fact that it's loading a local one like shown above. If you know how to change this, open an issue please. Anyways, to bypass this, we have to inform the `/boot/extlinux/extlinux.conf` where to locate our file. Change from

```conf
TIMEOUT 30
DEFAULT primary

MENU TITLE L4T boot options

LABEL primary
      MENU LABEL primary kernel
      LINUX /boot/Image
      INITRD /boot/initrd
      APPEND ${cbootargs} quiet root=/dev/mmcblk0p1 rw rootwait rootfstype=ext4 loglevel=7 console=ttyS0,115200n8 console=tty0 fbcon=map:0 net.ifnames=0
```

to

```conf
TIMEOUT 30
DEFAULT primary

MENU TITLE L4T boot options

LABEL primary
      MENU LABEL primary kernel
      LINUX /boot/Image
      INITRD /boot/initrd
      FDT /boot/tegra210-p3448-0000-p3449-0000-a00.dtb
      APPEND ${cbootargs} quiet root=/dev/mmcblk0p1 rw rootwait rootfstype=ext4 loglevel=7 console=ttyS0,115200n8 console=tty0 fbcon=map:0 net.ifnames=0
```

that is, add the path to your dtb file. In my case, `FDT /boot/tegra210-p3448-0000-p3449-0000-a00.dtb`.

In [105]:
ls -laF $DTB

-rw-r--r-- 1 root root 228845 11月  5 16:46 /boot/tegra210-p3448-0000-p3449-0000-a02.dtb


In [113]:
cat /boot/extlinux/extlinux.conf | sed '/^#/d' 

TIMEOUT 30
DEFAULT primary

MENU TITLE L4T boot options

LABEL primary
      MENU LABEL primary kernel
      LINUX /boot/Image
      INITRD /boot/initrd
      APPEND ${cbootargs} quiet root=/dev/mmcblk0p1 rw rootwait rootfstype=ext4 console=ttyS0,115200n8 console=tty0 fbcon=map:0 net.ifnames=0 





In [116]:
[ ! -f /boot/extlinux/extlinux.conf.orig ] && sudo cp -a /boot/extlinux/extlinux.conf{,.orig}

In [119]:
grep -q $DTB /boot/extlinux/extlinux.conf ||  sudo sed -i -e "/^[[:space:]]*INITRD \/boot\/initrd/a\      FDT $DTB" /boot/extlinux/extlinux.conf

In [120]:
cat /boot/extlinux/extlinux.conf | sed '/^#/d' 

TIMEOUT 30
DEFAULT primary

MENU TITLE L4T boot options

LABEL primary
      MENU LABEL primary kernel
      LINUX /boot/Image
      INITRD /boot/initrd
      FDT /boot/tegra210-p3448-0000-p3449-0000-a02.dtb
      APPEND ${cbootargs} quiet root=/dev/mmcblk0p1 rw rootwait rootfstype=ext4 console=ttyS0,115200n8 console=tty0 fbcon=map:0 net.ifnames=0 





Note that you can add a second testing profile, which can be selected at boot time if you have a serial device to plug into the jetson nano like in this video https://www.youtube.com/watch?v=Kwpxhw41W50. When you boot you can select your second `LABEL` by typing its number. This is useful if you want to test different `Image`s without substituting the original one like we did.

Now reboot, and then run

#### After reboot

In [11]:
REGEX2='kvm|^vhost|virt|ip[ _]?set|iscsi'

In [12]:
ls /dev | grep -iE "$REGEX2"
# =>
# kvm
# vhost-net
# vhost-vsock
# vsock

kvm
vhost-net


 to confirm if the `kvm` file exists. This means it's working. You should also run

In [13]:
ls /proc/device-tree/interrupt-controller
# Doc:
# => compatible '#interrupt-cells' interrupt-controller interrupt-parent interrupts linux,phandle name phandle reg status
# Before:
# => compatible  '#interrupt-cells'   interrupt-controller   interrupt-parent   linux,phandle   name   phandle   reg   status
# After:
# => compatible  '#interrupt-cells'   interrupt-controller   interrupt-parent   linux,phandle   name   phandle   reg   status
# => compatible  '#interrupt-cells'   interrupt-controller   interrupt-parent   linux,phandle   name   phandle   reg   status
# => compatible  '#interrupt-cells'   interrupt-controller   interrupt-parent   linux,phandle   name   phandle   reg   status

'#interrupt-cells'   interrupt-controller   linux,phandle   phandle   status
 compatible          interrupt-parent       name            reg


and see that the node `interrupts`, which didn't exist before, was added. This means the irc interrupt activation worked.

In [14]:
sudo dmesg | grep -iE "$REGEX2"

[    0.000000] Virtual kernel memory layout:
[    0.553913] gpiochip_setup_dev: registered GPIOs 0 to 255 on device: gpiochip0 (tegra-gpio)
[    0.607619] gpiochip_setup_dev: registered GPIOs 504 to 511 on device: gpiochip1 (max77620-gpio)
[    1.031164] kvm [1]: 8-bit VMID
[    1.031170] kvm [1]: IDMAP page: 84fb3000
[    1.031175] kvm [1]: HYP VA range: 4000000000:7fffffffff
[    1.033287] kvm [1]: Hyp mode initialized successfully
[    1.033363] kvm [1]: virtual timer IRQ7
[    1.676862] igbvf: Intel(R) Gigabit Virtual Function Network Driver - version 2.4.0-k


In [16]:
lsmod

Module                  Size  Used by
bnep                   19142  2
xt_conntrack            3979  1
ipt_MASQUERADE          2634  1
nf_nat_masquerade_ipv4     3993  1 ipt_MASQUERADE
nf_conntrack_netlink    33352  0
nfnetlink               9780  2 nf_conntrack_netlink
xt_addrtype             3979  2
iptable_filter          3008  1
iptable_nat             3423  1
nf_conntrack_ipv4      14222  2
nf_defrag_ipv4          2129  1 nf_conntrack_ipv4
nf_nat_ipv4             8176  1 iptable_nat
nf_nat                 25156  2 nf_nat_masquerade_ipv4,nf_nat_ipv4
nf_conntrack          124658  6 nf_conntrack_ipv4,nf_conntrack_netlink,nf_nat_masquerade_ipv4,xt_conntrack,nf_nat_ipv4,nf_nat
br_netfilter           17652  0
zram                   29753  4
overlay                55109  0
binfmt_misc            14703  1
userspace_alert         6833  0
nvgpu                1747340  18
ip_tables              21229  2 iptable_filter,iptable_nat
x_tables               40188  5 ip_tables,iptable_filter,ipt_MA

In [15]:
lsmod | grep -iE "$REGEX2"
# =>
# vhost_net              15023  0
# vhost                  52361  1 vhost_net
# macvtap                21473  1 vhost_net
# =>
# vhost_net              15023  0
# vhost                  52361  1 vhost_net
# macvtap                21473  1 vhost_net
# =>
# vhost_vsock            13434  0
# vmw_vsock_virtio_transport_common    30778  1 vhost_vsock
# vsock                  36419  2 vhost_vsock,vmw_vsock_virtio_transport_common
# vhost_net              15023  0
# vhost                  52489  2 vhost_vsock,vhost_net
# macvtap                21537  1 vhost_net

: 1

In [17]:
cat /etc/modules
# =>
# vhost_net

# /etc/modules: kernel modules to load at boot time.
#
# This file contains the names of kernel modules that should be loaded
# at boot time, one per line. Lines beginning with "#" are ignored.



You can run qemu/firecracker now. I only tested with firecracker though.

#### いらんかも

In [29]:
ls -lAF /lib/modules

total 8
drwxr-xr-x 3 root root 4096  5月  3 20:11 4.9.337/
drwxr-xr-x 3 root root 4096 12月 26 21:04 4.9.337-tegra/


In [30]:
sudo -S pwd < ~/.mypassword

/home/jetson/jetson_nano_kvm


In [31]:
# 8. オプションで、以下のコマンドでインストールされたカーネルモジュールをアーカイブできます：
sudo tar -C / --owner root --group root -cjf $L4T_ROOT/kernel/kernel_supplements.tbz2 /lib/modules

tar: Removing leading `/' from member names


In [35]:
mkdir -p $L4T_ROOT/kernel/
sudo mv kernel_supplements.tbz2 $L4T_ROOT/kernel/

In [36]:
# インストールされたモジュールは、ターゲットシステムの `/lib/modules/<kernel_version>` の内容として使用できます。
# `apply_binaries.sh` を実行する前に、展開されたリリースパッケージのkernelディレクトリにあるアーカイブを置き換えるために、このアーカイブを使用してください：
ls -l $L4T_ROOT/kernel/kernel_supplements.tbz2

-rw-r--r-- 1 root root 296113109  5月  3 22:49 /home/jetson/jetson_nano_kvm/Linux_for_Tegra/kernel/kernel_supplements.tbz2
